In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [2]:
target_file_dir = "../../datas/"
target_file_names = [
    "RealEstateData_20111_20164_13_main",
    "RealEstateData_20171_20234_13_main",
    "RealEstateData_20111_20234_14_main"
]
target_file_end = ".csv"
df_origin = pd.DataFrame()
for file_name in target_file_names:
    df = pd.read_csv(target_file_dir + file_name + target_file_end)
    df_origin = pd.concat([df_origin, df], axis=0)

In [3]:
print(df_origin.shape)
print(df_origin.columns)
print(df_origin.isnull().sum())

(451808, 16)
Index(['TradeNo', 'Type', 'Prefecture', 'Municipality', 'DistrictName',
       'Structure', 'FloorPlan', 'TradePrice', 'AgeAtTrade', 'Area',
       'TotalFloorArea', 'CoverageRatio', 'FloorAreaRatio', 'BuildingYearW',
       'TradeYear', 'TradeQuarter'],
      dtype='object')
TradeNo                0
Type                   0
Prefecture             0
Municipality           0
DistrictName         444
Structure          18304
FloorPlan         206699
TradePrice             0
AgeAtTrade         18208
Area                   0
TotalFloorArea    263472
CoverageRatio       6038
FloorAreaRatio      6038
BuildingYearW      18208
TradeYear              0
TradeQuarter           0
dtype: int64


In [10]:
# パラメータ
TradePriceThrethold = 15000
AgeAtTradeThrethold = 50
select_type = "中古マンション等" #宅地(土地と建物) 中古マンション等
# prefecture = "東京都"
use_cols = ["TradePrice","Area","TradeYear","AgeAtTrade", 'Municipality'] #Municipality Prefecture
category_cols = ['Municipality']
AreaThrethold = 300 if select_type == "宅地(土地と建物)" else 100 if select_type == "中古マンション等" else 0

# データ読み込み
col_choice_extract = (~df_origin.AgeAtTrade.isnull())&(df_origin.Type==select_type)#&(df_origin.Prefecture==prefecture)
col_choice_threthold = (df_origin.TradePrice<TradePriceThrethold)&(df_origin.Area<AreaThrethold)&(df_origin.AgeAtTrade<AgeAtTradeThrethold)
df_base = df_origin.loc[col_choice_extract&col_choice_threthold,use_cols].copy()
print(df_origin.shape[0], df_origin.loc[col_choice_extract,:].shape[0],df_base.shape[0])


451808 247102 238566


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

df = df_base.copy()
df = pd.get_dummies(df, columns=category_cols)

# 目的変数と説明変数に分割
y = df['TradePrice']
X = df.drop('TradePrice', axis=1)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレストモデルの構築
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# モデルの評価（MAE）
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print('Mean Absolute Error:', mae)


Mean Absolute Error: 862.1517844053536


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

df = df_base.copy()
df = pd.get_dummies(df, columns=category_cols)

# 目的変数と説明変数に分割
y = df['TradePrice']
X = df.drop('TradePrice', axis=1)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ランダムフォレストモデルの構築
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# モデルの評価（MAE）
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print('Mean Absolute Error:', mae)


Mean Absolute Error: 535.4138519512444


In [30]:
absolute_errors

16522      375.353845
169926    1737.085790
117885     459.130374
1634       181.699664
85649      517.536270
             ...     
48776     2377.028765
72788     1034.088544
32175      544.451059
10035       76.644985
19739       96.286612
Name: TradePrice, Length: 15569, dtype: float64